# Explosion Explorer

**If a bomb goes off in the forest, and noones around to confirm it, what's the probability that it really happened?**

**By Caleb Buffa and Zach Phillips**

**Explosion events recorded in databases are usually confirmed by humans. But what happens if nobody is around, or survives, to confirm the event? Is it possible to estimate the probability that a suspected explosion occurred?**

**The Explosion Explorer Application gives users the ability to explore unpopulated places and examine the effects that explosions have on the surface of the Earth. Explosion Explorer leverages explosion events from the open-source Armed Conflict Location Event Database (ACLED) and before/after changes in indices derived from publicly available Landsat and Sentinel-2 products (NDVI, EVI, NBRT, and SAR) as training data for estimating the probability that an explosion occurred in a certain area, during a certain time period. Explosions included in the training dataset include shellings/artillery/missiles, air/drone strikes, landmines, remote triggered devices, suicide bombings, and grenades.**

**Red points initially displayed on the map are the ACLED points for explosions. Explore a known explosion location, or select a location and date for an unconfirmed location to estimate the probability that an explosion occurred in the selected location near the selected date.**

**Steps to estimate unconfirmed explosions:**

1. Activate the Map by clicking the box above the `Submit` button.
2. Use the map tools to drop a placemarker on the map
3. Select a Date to Investigate
4. Click the `Submit` button to estimate whether/not your point has been subjected to any explosions on the date selected.


In [1]:
# Check geemap installation
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package is not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Import libraries
import os
import ee
import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl
import numpy as np
import ipyleaflet
import pickle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
import pandas as pd

ee.Initialize()

In [3]:
acled = pd.read_csv(r'2018-02-13-2021-02-17.csv')
acled_sub = acled.loc[(acled['sub_event_type'] == 'Shelling/artillery/missile attack')]
cols = ['longitude','latitude']
coordinates = acled_sub[cols].values.tolist()

In [ ]:
# Defining backend functions for widget

reducers = ee.Reducer.mean().combine(**
                                     {'reducer2': ee.Reducer.stdDev(), 
                                      'sharedInputs': True}).combine(**
                                                                     {'reducer2': ee.Reducer.max(), 
                                                                      'sharedInputs': True}).combine(**
                                                                                                     {'reducer2':ee.Reducer.min(), 
                                                                                                      'sharedInputs':True}).combine(**
                                                                                                                                    {'reducer2': ee.Reducer.percentile([25, 50, 75]), 
                                                                                                                                     'sharedInputs':True})
def det(im):
    return im.expression('b(0) * b(1)')
    
def chi2cdf(chi2, df):
    ''' Chi square cumulative distribution function for df degrees of freedom
    using the built-in incomplete gamma function gammainc() '''
    return ee.Image(chi2.divide(2)).gammainc(ee.Number(df).divide(2))

def sar(before, after, geometry): 
    m = 5
    
    try:
        # The observed test statistic image -2logq
        m2logq = det(before).log().add(det(after).log()).subtract(det(before.add(after)).log().multiply(2)).add(4*np.log(2)).multiply(-2*m)
        
        # The P value image prob(m2logQ > m2logq) = 1 - prob(m2logQ < m2logq).
        p_value = ee.Image.constant(1).subtract(chi2cdf(m2logq, 2))
        
        c_map = p_value.multiply(0).where(p_value.lt(0.05), 1)
        
        diff = after.subtract(before) # Getting the difference between the two images
        d_map = c_map.multiply(0)                    # Initialize the direction map to zero.
        d_map = d_map.where(det(diff).gt(0), 1)      # All pos or neg def diffs are now labeled 1.
        d_map = d_map.where(diff.select(0).gt(0), 2) # Re-label pos def (and label some indef) to 2.
        d_map = d_map.where(det(diff).lt(0), 1)      # Label all indef to 1.
        c_map = c_map.multiply(d_map) # Re-label the c_map, 0*X = 0, 1*1 = 1, 1*2= 2, 1*3 = 3.
        
        
        stats = c_map.reduceRegion(**{'reducer':reducers, 'bestEffort':True, 'scale':30, 'geometry':geometry}).getInfo()
        mean = stats['constant_mean']
        stddev = stats['constant_stdDev']
        p25 = stats['constant_p25']
        p50 = stats['constant_p50']
        p75 = stats['constant_p75']
        mini = stats['constant_min']
        maxi = stats['constant_max']
        
    except: 
        mean = None
        stddev = None
        p25 = None
        p50 = None
        p75 = None
        mini = None
        maxi = None
        
    return p25, p50, p75, mean, stddev, mini, maxi, c_map

def nbr(before, after, geometry):
    try:
        
        before = before.normalizedDifference(['B5', 'B7']).rename('NBR')
        after = after.normalizedDifference(['B5', 'B7']).rename('NBR')
        
        
        stats_b = before.reduceRegion(**{'reducer':reducers,'bestEffort':True, 'scale':30, 'geometry':geometry}).getInfo()    
        
        stats_a = after.reduceRegion(**{'reducer':reducers,'bestEffort':True, 'scale':30, 'geometry':geometry}).getInfo()

        mean_b = stats_b['NBR_mean']
        mean_a = stats_a['NBR_mean']

        p25_b = stats_b['NBR_p25']
        p25_a = stats_a['NBR_p25']

        p50_b = stats_b['NBR_p50']
        p50_a = stats_a['NBR_p50']

        p75_b = stats_b['NBR_p75']
        p75_a = stats_a['NBR_p75']

        stddev_b = stats_b['NBR_stdDev']
        stddev_a = stats_a['NBR_stdDev']

        min_b = stats_b['NBR_min']
        min_a = stats_a['NBR_min']

        max_b = stats_b['NBR_max']
        max_a = stats_a['NBR_max']

        p25 = p25_a - p25_b * 100
        p50 = p50_a - p50_b * 100
        p75 = p75_a - p50_b * 100
        mean = mean_a - mean_b * 100
        mini = min_a - min_b * 100
        maxi = max_a - max_b * 100
        stddev = stddev_a - stddev_b * 100
  
    except: 
        p25 = None
        p50 = None
        p75 = None
        mean = None
        stddev = None
        mini = None
        maxi = None
    
    return p25, p50, p75, mean, stddev, mini, maxi, before, after

def ndvi(before, after, geometry):
    
    try:
        
        before = before.normalizedDifference(['B5', 'B4']).rename('NDVI')
        
        after = after.normalizedDifference(['B5', 'B4']).rename('NDVI')
        
        stats_b = before.reduceRegion(**{'reducer':reducers,'bestEffort':True, 'scale':30, 'geometry':geometry}).getInfo()
        
        stats_a = after.reduceRegion(**{'reducer':reducers,'bestEffort':True, 'scale':30, 'geometry':geometry}).getInfo()

        mean_b = stats_b['NDVI_mean']
        mean_a = stats_a['NDVI_mean']

        p25_b = stats_b['NDVI_p25']
        p25_a = stats_a['NDVI_p25']

        p50_b = stats_b['NDVI_p50']
        p50_a = stats_a['NDVI_p50']

        p75_b = stats_b['NDVI_p75']
        p75_a = stats_a['NDVI_p75']

        stddev_b = stats_b['NDVI_stdDev']
        stddev_a = stats_a['NDVI_stdDev']

        min_b = stats_b['NDVI_min']
        min_a = stats_a['NDVI_min']

        max_b = stats_b['NDVI_max']
        max_a = stats_a['NDVI_max']

        p25 = p25_a - p25_b * 100
        p50 = p50_a - p50_b * 100
        p75 = p75_a - p50_b * 100
        mean = mean_a - mean_b * 100
        mini = min_a - min_b * 100
        maxi = max_a - max_b * 100
        stddev = stddev_a - stddev_b * 100 
  
    except: 
        p25 = None
        p50 = None
        p75 = None
        mean = None
        stddev = None
        mini = None
        maxi = None
    
    return p25, p50, p75, mean, stddev, mini, maxi, before, after

def evi(before, after, geometry):
    try:
        before_evi = before.expression(
            '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                'NIR': before.select('B5'),
                'RED': before.select('B4'),
                'BLUE': before.select('B2')})

        after_evi = after.expression(
            '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                'NIR': after.select('B5'),
                'RED': after.select('B4'),
                'BLUE': after.select('B2')})

        stats_b = before_evi.rename('EVI').reduceRegion(**{'reducer':reducers, 'bestEffort':True, 'scale':30, 'geometry':geometry}).getInfo()
        stats_a = after_evi.rename('EVI').reduceRegion(**{'reducer':reducers, 'bestEffort':True, 'scale':30, 'geometry':geometry}).getInfo()

        mean_b = stats_b['EVI_mean']
        mean_a = stats_a['EVI_mean']

        p25_b = stats_b['EVI_p25']
        p25_a = stats_a['EVI_p25']

        p50_b = stats_b['EVI_p50']
        p50_a = stats_a['EVI_p50']

        p75_b = stats_b['EVI_p75']
        p75_a = stats_a['EVI_p75']

        stddev_b = stats_b['EVI_stdDev']
        stddev_a = stats_a['EVI_stdDev']

        min_b = stats_b['EVI_min']
        min_a = stats_a['EVI_min']

        max_b = stats_b['EVI_max']
        max_a = stats_a['EVI_max']

        p25 = p25_a - p25_b * 100
        p50 = p50_a - p50_b * 100
        p75 = p75_a - p50_b * 100
        mean = mean_a - mean_b * 100
        mini = min_a - min_b * 100
        maxi = max_a - max_b * 100
        stddev = stddev_a - stddev_b * 100
  
    except: 
        p25 = None
        p50 = None
        p75 = None
        mean = None
        stddev = None
        mini = None
        maxi = None
    
    return p25, p50, p75, mean, stddev, mini, maxi, before_evi, after_evi

def difference(coordinate, date):
    
    point = ee.Geometry.Point(coordinate.getInfo()['features'][0]['geometry']['coordinates'])
    geometry = point.buffer(1000) # buffers point by 1 km

    md = ee.Date(date)
    sd = md.advance(-4, 'week')
    ed = md.advance(4, 'week')

    before_sar = ee.ImageCollection("COPERNICUS/S1_GRD_FLOAT").filterBounds(point).filterDate(sd, md).filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).limit(1, 'system:time_start', False).first() 
    after_sar = ee.ImageCollection("COPERNICUS/S1_GRD_FLOAT").filterBounds(point).filterDate(md, ed).filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).first()

        
    before_landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(point).filterDate(sd, md).sort('CLOUD_COVER').limit(1, 'system:time_start', False).first()
    after_landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(point).filterDate(md, ed).sort('CLOUD_COVER').first()
    
    ndvi_p25, ndvi_p50, ndvi_p75, ndvi_mean, ndvi_stddev, ndvi_min, ndvi_max, before_ndvi, after_ndvi = ndvi(before_landsat, 
                                                                                                             after_landsat,
                                                                                                             geometry)  

    evi_p25, evi_p50, evi_p75, evi_mean, evi_stddev, evi_min, evi_max, before_evi, after_evi = evi(before_landsat, 
                                                                                                   after_landsat, 
                                                                                                   geometry)

    sar_p25, sar_p50, sar_p75, sar_mean, sar_stddev, sar_min, sar_max, c_map = sar(before_sar,
                                                                                   after_sar, 
                                                                                   geometry)

    nbr_p25_, nbr_p50, nbr_p75, nbr_mean, nbr_stddev, nbr_min, nbr_max, before_nbr, after_nbr = nbr(before_landsat,
                                                                                                    after_landsat, 
                                                                                                    geometry)

    
    ar = np.array([ndvi_p25, ndvi_p50, ndvi_p75, ndvi_mean, ndvi_min, ndvi_max, 
                   evi_p25, evi_p50, evi_p75, evi_mean, evi_stddev, evi_min, evi_max, 
                   sar_mean, sar_stddev, sar_max, 
                   nbr_mean, nbr_stddev, nbr_p25_, nbr_p50, nbr_p75, nbr_min]).reshape(1,-1)
    
    try:
        pred = clf.predict(ar)
        prob = clf.predict_proba(ar)
        if pred[0] == 1:
            pred = f'Explosion Likely Occured On: {date}'
            prob = f"Probability of Explosion: {round(prob[0][1]*100, 2)}%"
        else:
            pred = f'Explosion Did Not Likely Occur On: {date}'
            prob = f'Probability of No Explosion: {round(prob[0][0]*100, 2)}%'
        print(f"{pred}\n{prob}")
    except:
        print('Error: Please choose different location/date')

    try:
        Map.addLayer(before_ndvi.clip(geometry), {'min':0, 'max':1, "palette": [
        'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
        '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
        '012E01', '011D01', '011301']}, 'Before NDVI')
    except:
        pass
    
    try:
        Map.addLayer(after_ndvi.clip(geometry), {'min':0, 'max':1, "palette": [
        'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
        '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
        '012E01', '011D01', '011301']}, 'After NDVI')
    except:
        pass
    
    try:
        Map.addLayer(before_evi.clip(geometry), {'min':0, 'max':1, 'palette': [
        'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
        '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
        '012E01', '011D01', '011301']}, 'Before EVI')
    except:
        pass
    
    try:
        Map.addLayer(after_evi.clip(geometry), {'min':0, 'max':1, "palette": [
        'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
        '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
        '012E01', '011D01', '011301']}, 'After EVI')
    except:
        pass
    
    try:
        Map.addLayer(before_nbr.clip(geometry), {'min':0, 'max':1, 'palette':['000000', 'FFFFFF']}, 'Before NBR')
    except:
        pass
    
    try:
        Map.addLayer(after_nbr.clip(geometry), {'min':0, 'max':1, 'palette':['000000', 'FFFFFF']}, 'After NBR')
    except:
        pass
    
    try:
        Map.addLayer(c_map.clip(geometry), {'palette':['white', 'blue', 'red']}, 'SAR Probability')
    except:
        pass
        
# Loading in trained classifier
filename = 'explosion_model.sav'
clf = pickle.load(open(filename, 'rb')) 

In [ ]:
# Create an interactive map
Map = geemap.Map(center=[0,0], zoom=2, add_google_map=False)
Map.add_basemap('HYBRID')

sentinel = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

# Adding date widget to map
date_picker = widgets.DatePicker(description='Pick a Date', disabled=False)
date_widget = widgets.Output(layout={'border': '1px solid black'})
date_control = ipyleaflet.WidgetControl(widget=date_widget, position='bottomright')
Map.add_control(date_control) 

with date_widget:
    display(date_picker)
    
prediction_widget = widgets.Output(layout={'border': '1px solid black'})
prediction_control = ipyleaflet.WidgetControl(widget=prediction_widget, position='topright')
Map.add_control(prediction_control)

style = {'description_width': 'initial'}

submit = widgets.Button(description='Submit',button_style='primary',
                        tooltip='Click me',style=style)

aoi_widget = widgets.Checkbox(value=False, 
                              description='Drop point, pick date, and click Submit',style=style)
full_widget = widgets.VBox([widgets.HBox([aoi_widget]),submit])

full_control = ipyleaflet.WidgetControl(widget=full_widget, position='bottomright')
Map.add_control(full_control)

def submit_clicked(b):
    with prediction_widget:
        prediction_widget.clear_output()
        print('Computing...')
#        Map.default_style = {'cursor': 'wait'}
        try:
            roi = ee.FeatureCollection(Map.draw_last_feature)
            date = str(date_picker.value)           
            if roi:
                if Map.draw_last_feature is not None:
                    roi = ee.FeatureCollection(Map.draw_last_feature)                    
                    if date_picker.value is not None:
                        difference(roi, date)
                    else:
                        pass                    
                else:
                    pass                
            else:
                pass        
        except Exception as e:
            print(e)
            print('An Error Occured During Computation')
submit.on_click(submit_clicked)

cords = []
for c in coordinates:    
    point = ee.Geometry.Point(c[0], c[1])
    cords.append(point)
fc = ee.FeatureCollection(cords)
Map.addLayer(fc, {'color':'Red'}, 'ACLED Explosions')

Map